In [1]:
import os
import findspark


from pyspark import SparkContext
from pyspark.streaming import StreamingContext

findspark.init()


In [2]:
sc = SparkContext(appName='Window Operation')
ssc = StreamingContext(sparkContext=sc, batchDuration=30)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 16:06:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [3]:
def reduce_by_key(rdd):
    return rdd.reduceByKey(lambda a, b: a + b)

socket_stream = ssc.socketTextStream(hostname='127.0.0.1', port=5555)
words_pairs = socket_stream.flatMap(lambda l: l.split(" ")).map(lambda word: (word, 1)).transform(reduce_by_key)


In [4]:
# when the window starts (t=0) looks for data from t=-60 -> t=0
#
#   ***** slideDuration must be multiple of the parent dstream's slide (batch) duration *****
#   ***** windowDuration must be multiple of the parent dstream's slide (batch) duration *****

words_pairs.window(windowDuration=120, slideDuration=60).pprint()

In [5]:
ssc.start()


In [6]:
ssc.awaitTerminationOrTimeout(timeout=200)

24/03/26 16:06:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/03/26 16:06:31 WARN BlockManager: Block input-0-1711465591600 replicated to only 0 peer(s) instead of 1 peers
24/03/26 16:06:37 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/03/26 16:06:37 WARN BlockManager: Block input-0-1711465597600 replicated to only 0 peer(s) instead of 1 peers
24/03/26 16:06:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/03/26 16:06:39 WARN BlockManager: Block input-0-1711465599400 replicated to only 0 peer(s) instead of 1 peers


-------------------------------------------
Time: 2024-03-26 16:07:00
-------------------------------------------
('b', 2)
('a', 3)
('c', 1)



-------------------------------------------
Time: 2024-03-26 16:08:00
-------------------------------------------
('b', 2)
('a', 3)
('c', 1)



24/03/26 16:08:18 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/03/26 16:08:18 WARN BlockManager: Block input-0-1711465698600 replicated to only 0 peer(s) instead of 1 peers


-------------------------------------------
Time: 2024-03-26 16:09:00
-------------------------------------------
('v', 2)



ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hso/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hso/.pyenv/versions/3.11.0/lib/python3.11/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
24/03/26 16:09:29 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.S

KeyboardInterrupt: 